In [3]:
# Imports
from aalpy.SULs import MealySUL
from aalpy.oracles import StatePrefixEqOracle

from DataProcessing import get_coffee_machine, generate_data_from_automaton
from TrainAndExtract import train_RNN_on_mealy_data, extract_finite_state_transducer

In [4]:
# Get coffee machine model
coffee_machine = get_coffee_machine()

# Get its input and output alphabet
input_al = coffee_machine.get_input_alphabet()
output_al = {output for state in coffee_machine.states for output in state.output_fun.values()}


In [5]:
# Generate training data from coffee machine FSM
train_seq, train_labels = generate_data_from_automaton(coffee_machine, input_al,
                                                       num_examples=10000, lens=(2, 5, 8, 10))

training_data = (train_seq, train_labels)


In [6]:
# Train 2 RNNs with the data
rnn_1 = train_RNN_on_mealy_data(coffee_machine, data=training_data, ex_name='coffee_1')
rnn_2 = train_RNN_on_mealy_data(coffee_machine, data=training_data, ex_name=f'coffee_2')

Starting train
Epoch 0: Accuracy 0.96289, Avg. Loss 8.93819 Validation Accuracy 0.9575
Epoch 1: Accuracy 0.96602, Avg. Loss 3.93633 Validation Accuracy 0.9585
Epoch 2: Accuracy 0.97726, Avg. Loss 2.88321 Validation Accuracy 0.9765
Epoch 3: Accuracy 0.98363, Avg. Loss 2.01579 Validation Accuracy 0.983
Epoch 4: Accuracy 0.98326, Avg. Loss 1.97149 Validation Accuracy 0.9815
Epoch 5: Accuracy 0.98551, Avg. Loss 1.7448 Validation Accuracy 0.982
Epoch 6: Accuracy 0.98876, Avg. Loss 1.46309 Validation Accuracy 0.9885
Epoch 7: Accuracy 0.99475, Avg. Loss 1.00747 Validation Accuracy 0.9955
Epoch 8: Accuracy 0.99925, Avg. Loss 0.42133 Validation Accuracy 0.9985
Epoch 9: Accuracy 0.99988, Avg. Loss 0.11914 Validation Accuracy 1.0
Epoch 10: Accuracy 1.0, Avg. Loss 0.05698 Validation Accuracy 1.0
Epoch 11: Accuracy 1.0, Avg. Loss 0.02926 Validation Accuracy 1.0
Epoch 12: Accuracy 1.0, Avg. Loss 0.01781 Validation Accuracy 1.0
Done training!
Starting train
Epoch 0: Accuracy 0.95815, Avg. Loss 9.0655

In [7]:
# Extract automata from each RNN
learned_automaton_1 = extract_finite_state_transducer(rnn_1, input_al, output_al, max_learning_rounds=25)
learned_automaton_2 = extract_finite_state_transducer(rnn_2, input_al, output_al, max_learning_rounds=25)

Hypothesis 1: 2 states.
Hypothesis 2: 5 states.
Hypothesis 3: 6 states.
Hypothesis 4: 7 states.
Hypothesis 5: 8 states.
Hypothesis 6: 10 states.
Hypothesis 7: 13 states.
Hypothesis 8: 16 states.
Hypothesis 9: 18 states.
Hypothesis 10: 21 states.
Hypothesis 11: 22 states.
Hypothesis 12: 24 states.
Hypothesis 13: 25 states.
Hypothesis 14: 26 states.
Hypothesis 15: 27 states.
Hypothesis 16: 46 states.
Hypothesis 17: 48 states.
Hypothesis 18: 49 states.
Hypothesis 19: 54 states.
Hypothesis 20: 58 states.
Hypothesis 21: 60 states.
Hypothesis 22: 64 states.
Hypothesis 23: 68 states.
Hypothesis 24: 71 states.
Hypothesis 25: 76 states.
-----------------------------------
Learning Finished.
Learning Rounds:  26
Number of states: 76
Time (in seconds)
  Total                : 6.84
  Learning algorithm   : 5.15
  Conformance checking : 1.69
Learning Algorithm
 # Membership Queries  : 8965
 # Steps               : 143430
Equivalence Query
 # Membership Queries  : 1449
 # Steps               : 41860

In [11]:
sul = MealySUL(learned_automaton_2)
sul2 = MealySUL(learned_automaton_1)

# Define eq. oracle used for conformance-checking (learning-based testing)
eq_oracle = StatePrefixEqOracle(input_al, sul, walks_per_state=100, walk_len=20)


In [12]:
# Find and print cases of non-conformance
cex_set = set()
for i in range(200):
    cex = eq_oracle.find_cex(learned_automaton_1)
    if cex:
        if tuple(cex) not in cex_set:
            print('--------------------------------------------------------------------------')
            print('Case of Non-Conformance between Automata: ', cex)
            print('Model 1  : ', sul.query(cex))
            print('Model 2  : ', sul2.query(cex))
        cex_set.add(tuple(cex))


 'star', 'star', 'star']
--------------------------------------------------------------------------
Case of Non-Conformance between Automata:  ('pod', 'water', 'water', 'water', 'water', 'water', 'button', 'clean', 'water', 'water', 'pod', 'pod', 'water', 'water', 'water', 'button', 'water')
Model 1  :  ['check', 'check', 'check', 'check', 'check', 'check', 'coffee', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'coffee', 'coffee']
Model 2  :  ['check', 'check', 'check', 'check', 'check', 'check', 'coffee', 'check', 'star', 'star', 'star', 'star', 'star', 'check', 'check', 'coffee', 'star']
--------------------------------------------------------------------------
Case of Non-Conformance between Automata:  ('pod', 'water', 'water', 'water', 'water', 'water', 'water', 'button', 'clean', 'pod', 'pod', 'water', 'clean')
Model 1  :  ['check', 'check', 'check', 'check', 'check', 'check', 'check', 'coffee', 'check', 'check', 'check', 'check', 'check']
Model 2  :  ['